In [ ]:
# Generate new token with credential and secret key.
# Note: Onet time password will be generated by the funtion itself from secret key

def generate_token(username, password, secret):
    import requests
    import json
    import getpass    
    import pyotp
    
    username = input("Enter Your UserName: ")              # Enter UserName
    password = getpass.getpass("Enter Your Password: ")    # Enter Password
    
    # Generate one time password
    
    secret = getpass.getpass("Enter Your Secret Key")      # Enter Secret Key 
    OTP_NOW = 0
    
    topt = pyotp.TOTP(secret)
    
    if topt.verify(OTP_NOW) == False:
        OTP_NOW = topt.now()            # Get OTP from Generate Token function         
    else:
        return ("Retry with valid secret key")

    # Generate Token for consuming api
    
    auth_payload = {"login":username, "password":password, "verificationCode":OTP_NOW}
    api_endpoint = "https://api.abc.com/partner/v2/login" # api login url
    head = {'content-type':'application/json'}
    
    generate_token = requests.post(url = api_endpoint, data = json.dumps(auth_payload), headers = head)
    
    # Validate api response
    if generate_token.status_code == 200:
        token = generate_token.json()['token'] #New Token
        refresh_token = generate_token.json()['refreshToken'] # Refresed Token
        return token, refresh_token
    else:
        print ("Incorrect Auth Credentials. Please Retry")
    

In [ ]:
token, refresh_token  = generate_token('username', 'password', 'secret')

In [ ]:
generate_token('username', 'password', 'secret')

In [ ]:
# If the token is expired refresh token via generate_refToken function

def generate_refToken():
    import requests
    import json
    
    refresh_api_endpoint = 'https://api.abc.com/partner/v1/login/refresh'
    ref_payload = {'refreshToken':refresh_token}  #refresh token is fetched from generate_refresh function
    head = {'content-type':'application/json'}
    
    ref_token = requests.post(url = refresh_api_endpoint, data = json.dumps(ref_payload), headers = head)
    
    if ref_token.status_code == 200:
        token_1 = ref_token.json()['token']
        refresh_token_1 = ref_token.json()['refreshToken']
        return token_1, refresh_token_1
    else:
        print("Incorrect credentials. Please generate new token") 

In [ ]:
generate_refToken()

In [ ]:
# Get profiles from the api
global_UserIDs = []
global_page = 0


def get_profiles():
    
    global global_UserIDs
    global global_page
    
    import requests
    from datetime import datetime
    from pytz import timezone
    
    
    profile_endpoint = 'https://api.abc.com/partner/v1/users/profiles'
    
    session_validate = requests.Session()
    
    token_status = session_validate.get(url = profile_endpoint,
                                        params = {'accessCodeType':'CONSUMER', 
                                                  'sinceDate' : datetime.now(),
                                                  'limit' : 1}, 
                                        headers = {'x-session-token': live_token}) 
          
    while token_status.status_code == 200:  # Validating token
        tz = timezone('EST')
        time_stamp = datetime.now(tz).strftime("%Y-%m-%d") #Since date (Central time zone)
        limit = 10

        session = requests.Session()

        token = live_token  # Get the live token from "live_token" 
        data = {'x-session-token': live_token}
        payload = {'accessCodeType':'CONSUMER', 
                   'sinceDate' : time_stamp, 
                   'page' : global_page,
                   'limit' : limit}
    
        r_profile = session.get(url = profile_endpoint,
                                params = payload, 
                                headers = data)
        for i in r_profile.json():
            
            with open('profile_'+str(i['userId'])+'.json', 'w') as json_file:
                json_file.write(str(r_profile.json()))
            global_UserIDs.append(i['userId'])
        global_page += 1

In [ ]:
# Check for partial break and last record fetched for the date
get_profiles()

In [ ]:
# Get profiles from the api increment date
global_UserIDs = []
global_page = 0


def get_profiles():
    
    global global_UserIDs
    global global_page
    
    import requests
    from datetime import datetime
    from pytz import timezone
    
    
    profile_endpoint = 'https://api.abc.com/partner/v1/users/profiles'
    
    session_validate = requests.Session()
    
    token_status = session_validate.get(url = profile_endpoint,
                                        params = {'accessCodeType':'CONSUMER', 
                                                  'sinceDate' : datetime.now(),
                                                  'limit' : 1}, 
                                        headers = {'x-session-token': live_token}) 
          
    while token_status.status_code == 200:  # Validating token
        tz = timezone('CST')
        #time_stamp = #Since date (Central time zone)
        
        date = datetime.datetime(2015,9,25,0,0,0)
        for i in range(5):
            time_stamp += datetime.timedelta(days=1)

        #limit = 10

        session = requests.Session()

        token = live_token  # Get the live token from "live_token" 
        data = {'x-session-token': live_token}
        payload = {'accessCodeType':'CONSUMER', 
                   'sinceDate' : time_stamp
#                  ,'page' : global_page
#                  ,'limit' : limit
                  }
    
        r_profile = session.get(url = profile_endpoint,
                                params = payload, 
                                headers = data)
        for i in r_profile.json():
            
            with open('profile_'+str(i['userId'])+'.json', 'w') as json_file:
                json_file.write(str(r_profile.json()))
            global_UserIDs.append(i['userId'])
        global_page += 1

In [ ]:
get_profiles()

In [ ]:
# AVRO schema in python
# https://fastavro.readthedocs.io/en/latest/

In [ ]:
#Define AVRO Schema
schema = {
     "type": "record",
     "namespace": "com.example",
     "name": "FullName",
     "fields": [
       { "name": "first", "type": "string" },
       { "name": "last", "type": "string" }
     ]
}

In [ ]:
noUserData_AVRO = []
parsed_schema = parse_schema(schema)
for user in userIDs:
    payload = {'userId' : user}
    r_profile = requests.get('https://api.abc.com/partner/v1/users/profiles', params=payload, headers = data)
    if r_profile.status_code == 200:
        record = r_profile.json()
        with open('profile_'+str(user)+'.avro', 'wb') as out_avro:
            writer(out_avro, parsed_schema, record)
    else:
        noUserData_AVRO.append(user)

In [ ]:
# View avro files
with open('profile_114703.avro', 'rb') as fo:
    for record in reader(fo):
        print(record)